In [ ]:
import plotly.express as px
import pickle
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

with open('drive/MyDrive/POCS FP/annual_counts_by_state.pkl', 'rb') as pickled_tweets:
    counts_by_state = pickle.load(pickled_tweets)

pickled_tweets.close()

counts_by_state = counts_by_state.reset_index(drop=True)



In [ ]:
def choro_map(year):
    counts_by_state[year] = pd.to_numeric(counts_by_state[year])
    
    fig = px.choropleth(data_frame=counts_by_state,
                        locations='states', 
                        locationmode="USA-states", 
                        scope="usa",
                        color=year,
                        color_continuous_scale = px.colors.sequential.amp,
                        )
    
    fig.show()

In [ ]:
choro_map('2010')

In [ ]:
#@title { run: "auto" }
year = 2011 #@param {type:"slider", min:2010, max:2018}
choro_map(str(year))

NameError: ignored

In [ ]:
def choro_map_animated():
    years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018']
    counts_by_state[year] = pd.to_numeric(counts_by_state[year])
    
    fig = px.choropleth(data_frame=counts_by_state,
                        locations='states', 
                        locationmode="USA-states", 
                        scope="usa",
                        color=year,
                        color_continuous_scale = px.colors.sequential.amp,
                        animation_frame=years
                        )
    
    fig.show()

In [ ]:
choro_map_animated()

KeyError: ignored